In [4]:
import torch
import pandas as pd
import torch
import torch.nn as nn
class MatrixFactorization(nn.Module):
    def __init__(self, n_users, n_anime, n_factors):
        super(MatrixFactorization, self).__init__()
        self.user_factors = nn.Embedding(n_users, n_factors)
        self.anime_factors = nn.Embedding(n_anime, n_factors)

    def forward(self, user, anime):
        return (self.user_factors(user) * self.anime_factors(anime)).sum(1)
# Load the model
mapping_dict = torch.load('mapping.pth')
user_mapping = mapping_dict['user_mapping']
anime_mapping = mapping_dict['anime_mapping']
n_factors=100
model = MatrixFactorization(len(user_mapping), len(anime_mapping), n_factors)
model.load_state_dict(torch.load('model.pth'))
model.eval()

# Load mappings and DataFrame

df_anime = pd.read_csv('C:/Users/vigne/OneDrive/Documents/WPI/Sem_2/IR/project/archive/animes.csv')
df_anime = df_anime[['uid', 'title', 'genre']].rename(columns={'uid': 'anime_id'})

# Get the user's preferred genre
# Get the user's preferred genres
user_genres_input = input("Enter your preferred genres separated by commas: ")
user_genres = [genre.strip() for genre in user_genres_input.split(',')]

# Find not watched animes
not_watched = [anime_idx for anime_idx in range(len(anime_mapping))]

# Get scores for not watched anime
user_tensor = torch.tensor([len(user_mapping) - 1] * len(not_watched), dtype=torch.long)
anime_tensor = torch.tensor(not_watched, dtype=torch.long)
anime_scores = model(user_tensor, anime_tensor)

# Get top N recommended anime indices and their respective anime IDs
N = 50
top_N_indices = torch.topk(anime_scores, N).indices
top_N_anime_ids = [list(anime_mapping.keys())[list(anime_mapping.values()).index(idx)] for idx in top_N_indices.tolist()]

# Filter the top N recommended anime titles based on the user's preferred genres
recommended_anime = df_anime[df_anime['anime_id'].isin(top_N_anime_ids) & df_anime['genre'].apply(lambda x: any(g.lower() in x.lower() for g in user_genres) if isinstance(x, str) else False)].reset_index(drop=True)

# Display the filtered recommended anime titles
print(f"Recommended Anime in {', '.join(user_genres)} genres:")
print(recommended_anime)



Enter your preferred genres separated by commas: Comedy
Recommended Anime in Comedy genres:
    anime_id                                              title  \
0        535                                       Slayers Next   
1       1486                              Kodomo no Omocha (TV)   
2      10301                         Rio: Rainbow Gate! Special   
3      30155      Hoshi no Kirby:  Taose!! Koukaku Majuu Ebizou   
4      16183                                     Lupin Shanshei   
5       1920                   Urusei Yatsura Movie 1: Only You   
6      10302     Pocket Monsters: Diamond & Pearl Tokubetsu-hen   
7        127                                       Gate Keepers   
8        691                                            Yawara!   
9       1804                                 Koko wa Green Wood   
10       535                                       Slayers Next   
11      1486                              Kodomo no Omocha (TV)   
12      7255                         